In [14]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import re

from vietnamadminunits.parser.utils import key_normalize

import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path().resolve().parent.parent

In [15]:
# IDEA
data = {
    'thanhphohochiminh': {'keywords': ['thanhphohochiminh', 'hochiminh', 'hcm']},
    'hanoi': {'keywords': ['hanoi', 'hn']},
}

# Bước 1: Tạo list keyword, sắp xếp theo độ dài giảm dần
keywords = sorted(
    [kw for v in data.values() for kw in v['keywords']],
    key=len,
    reverse=True
)

# Bước 2: Tạo regex pattern
pattern = re.compile('|'.join(re.escape(k) for k in keywords), flags=re.IGNORECASE)

# Bước 3: Kiểm tra chuỗi address
address = 'quan5, hcm'

match = pattern.search(address)
keyword = match.group(0) if match else None

# Bước 4: Tìm key trong data tương ứng với keyword
data_key = next((k for k, v in data.items() if keyword and keyword.lower() in [kw.lower() for kw in v['keywords']]), None)

print(data_key)

thanhphohochiminh


In [16]:
def create_sort(text, level=1):
    if isinstance(text, str):
        if level == 1:
            text = re.sub(r'^Tỉnh\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        elif level == 2:
            if re.search(r'^Quận\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Quận\s|Huyện\s|Thị xã\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        else:
            if re.search(r'^Phường\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Phường\s|Thị trấn\s|Xã\s', '', text, flags=re.IGNORECASE)

        return text.strip()
    return text

district_type_acronym = {
    'Quận': 'q',
    'Thị xã': 'tx',
    'Thành phố': 'tp',
    'Huyện': 'h',
}
ward_type_acronym = {
    'Phường': 'p',
    'Thị trấn': 'tt',
    'Xã': 'x'
}
def create_keywords(row, level=1):
    keywords = []
    if level == 1:
        keywords.append(row['provinceKey'])
        keywords.append(row['provinceShortKey'])
        if pd.notnull(row['provinceAlias']):
            aliases = json.loads(row['provinceAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    elif level == 2:
        keywords.append(row['districtKey'])
        if not row['districtShortKeyDuplicated']:
            keywords.append(row['districtShortKey'])
        else:
            keywords.append(key_normalize(f"{row['districtShortKey']} {row['districtType']}"))
            keywords.append(key_normalize(f"{district_type_acronym[row['districtType']]} {row['districtShortKey']}"))

        if pd.notnull(row['districtAlias']):
            aliases = json.loads(row['districtAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

        if re.search(r'^quan\d{1,2}', row['districtKey'], flags=re.IGNORECASE):
            keywords.append(row['districtKey'].replace('quan', 'q'))
            keywords.append(row['districtKey'].replace('quan', 'q.'))
            keywords.append(row['districtKey'].replace('quan', 'district'))

    else:
        if pd.notnull(row['wardKey']):
            keywords.append(row['wardKey'])
            if not row['wardShortKeyDuplicated']:
                keywords.append(row['wardShortKey'])
            else:
                keywords.append(key_normalize(f"{row['wardShortKey']} {row['wardType']}"))
                keywords.append(key_normalize(f"{ward_type_acronym[row['wardType']]} {row['wardShortKey']}"))

            if pd.notnull(row['wardAlias']):
                aliases = json.loads(row['wardAlias'])
                for a in aliases:
                    keywords.append(key_normalize(a))

            if re.search(r'^phuong\d{1,2}', row['wardKey'], flags=re.IGNORECASE):
                keywords.append(row['wardKey'].replace('phuong', 'p'))
                keywords.append(row['wardKey'].replace('phuong', 'p.'))
                keywords.append(row['wardKey'].replace('phuong', 'ward'))
        else:
            return np.nan

    keywords = list(set(keywords))
    keywords = sorted(keywords, key=len, reverse=True)
    return json.dumps(keywords)

In [17]:
df_convert = pd.read_csv(BASE_DIR / 'data/interim/danhmuc_and_sapnhap_has_default_new_ward.csv')
cols = [
    'provinceCode',
    'districtCode',
    'districtType',
    'wardCode',
    'wardType',
    'province',
    'district',
    'ward'
]
df = df_convert[cols].drop_duplicates().reset_index(drop=True)

In [18]:
# ENRICH DATA
unit_cols = ['province', 'district', 'ward']
level_map = {
    'province': 1,
    'district': 2,
    'ward': 3
}

for col in unit_cols:
    # Create short version
    level = level_map[col]
    df[f"{col}Short"] = df[col].apply(create_sort, args=(level,))

    # Create key
    df[f"{col}Key"] = df[f"{col}"].apply(key_normalize)

    # Create short key
    df[f"{col}ShortKey"] = df[f"{col}Short"].apply(key_normalize)

In [19]:
# -- CREATE ALIAS
# Khởi tạo cột alias rỗng
for col in ['province', 'district', 'ward']:
    df[f"{col}Alias"] = np.nan

# Province alias data
province_alias_data = {
    'thanhphohanoi': ['hn'],
    'thanhphohochiminh': ['hcm'],
    'tinhbariavungtau': ['baria', 'vungtau'],
}

# District alias data (theo từng province)
district_alias_data = {
    'thanhphohochiminh': {
        'thanhphothuduc': ['quan9', 'quan2', 'q9', 'q2', 'q.9', 'q.2']
    }
}

# Ward alias data (theo từng province > district > ward)
ward_alias_data = {
    # 'thanhphohochiminh': {
    #     'thanhphothuduc': {
    #         'phuongtruongthanh': ['truongthanh', 'p.truongthanh']
    #     }
    # }
}

# Gán provinceAlias
for key, value in province_alias_data.items():
    df.loc[df['provinceKey'] == key, 'provinceAlias'] = json.dumps(value)

# Gán districtAlias
for province_key, district_data in district_alias_data.items():
    for district_key, value in district_data.items():
        df.loc[
            (df['provinceKey'] == province_key) & (df['districtKey'] == district_key),
            'districtAlias'
        ] = json.dumps(value)

# Gán wardAlias
for province_key, district_data in ward_alias_data.items():
    for district_key, ward_data in district_data.items():
        for ward_key, value in ward_data.items():
            df.loc[
                (df['provinceKey'] == province_key) &
                (df['districtKey'] == district_key) &
                (df['wardKey'] == ward_key),
                'wardAlias'
            ] = json.dumps(value)

In [20]:
# CHECK DUPLICATED FOR DISTRICT
df_district = df[['province', 'provinceKey', 'district', 'districtKey', 'districtShortKey']].drop_duplicates()

# districtKey
df_district.groupby(['province', 'districtKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False).head()
# districtKey is unique

,province,districtKey,count
0,Thành phố Cần Thơ,huyencodo,1
467,Tỉnh Quảng Nam,huyenphuninh,1
459,Tỉnh Quảng Nam,huyenbactramy,1
460,Tỉnh Quảng Nam,huyendailoc,1
461,Tỉnh Quảng Nam,huyendonggiang,1


In [21]:
# districtShortKey
count_district_short_key = df_district.groupby(['province', 'districtShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_district_short_key = count_district_short_key[count_district_short_key['count']>1].copy()
duplicated_district_short_key['districtShortKeyDuplicated'] = True
duplicated_district_short_key.drop(columns=['count'], inplace=True)
duplicated_district_short_key

,province,districtShortKey,districtShortKeyDuplicated
577,Tỉnh Tiền Giang,cailay,True
292,Tỉnh Hậu Giang,longmy,True
254,Tỉnh Hà Tĩnh,kyanh,True
682,Tỉnh Đồng Tháp,hongngu,True
680,Tỉnh Đồng Tháp,caolanh,True
590,Tỉnh Trà Vinh,duyenhai,True


In [22]:
# Add flag for districtShortKey
df = pd.merge(df, duplicated_district_short_key, on=['province', 'districtShortKey'], how='left')
df['districtShortKeyDuplicated'].fillna(False, inplace=True)

In [23]:
# CHECK DUPLICATED WARD
count_ward_key = df.groupby(['province', 'district', 'wardKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
count_ward_key['wardKeyDuplicated'] = np.where(count_ward_key['count']>1, True, False)
duplicated_ward_key = count_ward_key[count_ward_key['wardKeyDuplicated']]
duplicated_ward_key.drop(columns=['count'], inplace=True)
duplicated_ward_key

,province,district,wardKey,wardKeyDuplicated
5357,Tỉnh Nam Định,Huyện Trực Ninh,xatruchung,True
7586,Tỉnh Thanh Hóa,Huyện Hậu Lộc,xahoaloc,True
6760,Tỉnh Quảng Ngãi,Huyện Bình Sơn,xabinhthanh,True
7551,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangthanh,True
6542,Tỉnh Quảng Nam,Huyện Nam Trà My,xatradon,True
7546,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangquy,True
7545,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangphu,True
5533,Tỉnh Nghệ An,Huyện Kỳ Sơn,xanamcan,True
1295,Tỉnh An Giang,Huyện Phú Tân,xaphuthanh,True
5611,Tỉnh Nghệ An,Huyện Quỳ Châu,xachaubinh,True


In [24]:
# Add flag for wardKey
df = pd.merge(df, duplicated_ward_key, on=['province', 'district', 'wardKey'], how='left')
df['wardKeyDuplicated'].fillna(False, inplace=True)

# Change wardKey and wardShortKey to accented key
df['wardKey'] = np.where(df['wardKeyDuplicated'], df['ward'].apply(key_normalize, args=([], False)), df['wardKey'])
df['wardShortKey'] = np.where(df['wardKeyDuplicated'], df['wardShort'].apply(key_normalize, args=([], False)), df['wardShortKey'])

In [25]:
# wardShortKey
count_ward_short_key = df.groupby(['province', 'district', 'wardShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_short_key = count_ward_short_key[count_ward_short_key['count']>1].copy()
duplicated_ward_short_key['wardShortKeyDuplicated'] = True
duplicated_ward_short_key.drop(columns=['count'], inplace=True)
duplicated_ward_short_key

,province,district,wardShortKey,wardShortKeyDuplicated
8685,Tỉnh Trà Vinh,Huyện Trà Cú,dinhan,True
286,Thành phố Hà Nội,Huyện Gia Lâm,yenvien,True
30,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,thanhan,True
6945,Tỉnh Quảng Ninh,Huyện Đầm Hà,damha,True
7215,Tỉnh Sóc Trăng,Huyện Long Phú,longphu,True
4749,Tỉnh Long An,Huyện Đức Hòa,hiephoa,True
4496,Tỉnh Lai Châu,Huyện Mường Tè,muongte,True
7252,Tỉnh Sóc Trăng,Huyện Trần Đề,lichhoithuong,True
2770,Tỉnh Cà Mau,Huyện Thới Bình,thoibinh,True
4708,Tỉnh Long An,Huyện Tân Thạnh,tanthanh,True


In [26]:
# Add flag for wardShortKey
df = pd.merge(df, duplicated_ward_short_key, on=['province', 'district', 'wardShortKey'], how='left')
df['wardShortKeyDuplicated'].fillna(False, inplace=True)

In [40]:
df[df['wardShortKeyDuplicated']]

,provinceCode,districtCode,districtType,wardCode,wardType,province,district,ward,provinceShort,provinceKey,...,wardShortKey,provinceAlias,districtAlias,wardAlias,districtShortKeyDuplicated,wardKeyDuplicated,wardShortKeyDuplicated,provinceKeywords,districtKeywords,wardKeywords
165,1,18,Huyện,526.0,Thị trấn,Thành phố Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Hà Nội,thanhphohanoi,...,yenvien,"[""hn""]",NaN,NaN,False,False,True,"[""thanhphohanoi"", ""hanoi"", ""hn""]","[""huyengialam"", ""gialam""]","[""yenvienthitran"", ""thitranyenvien"", ""ttyenvien""]"
167,1,18,Huyện,532.0,Xã,Thành phố Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Hà Nội,thanhphohanoi,...,yenvien,"[""hn""]",NaN,NaN,False,False,True,"[""thanhphohanoi"", ""hanoi"", ""hn""]","[""huyengialam"", ""gialam""]","[""yenvienxa"", ""xayenvien"", ""xyenvien""]"
1425,12,107,Huyện,3433.0,Thị trấn,Tỉnh Lai Châu,Huyện Mường Tè,Thị trấn Mường Tè,Lai Châu,tinhlaichau,...,muongte,NaN,NaN,NaN,False,False,True,"[""tinhlaichau"", ""laichau""]","[""huyenmuongte"", ""muongte""]","[""thitranmuongte"", ""muongtethitran"", ""ttmuongte""]"
1430,12,107,Huyện,3445.0,Xã,Tỉnh Lai Châu,Huyện Mường Tè,Xã Mường Tè,Lai Châu,tinhlaichau,...,muongte,NaN,NaN,NaN,False,False,True,"[""tinhlaichau"", ""laichau""]","[""huyenmuongte"", ""muongte""]","[""xamuongte"", ""muongtexa"", ""xmuongte""]"
1648,14,125,Huyện,4105.0,Thị trấn,Tỉnh Sơn La,Huyện Mai Sơn,Thị trấn Hát Lót,Sơn La,tinhsonla,...,hatlot,NaN,NaN,NaN,False,False,True,"[""tinhsonla"", ""sonla""]","[""huyenmaison"", ""maison""]","[""thitranhatlot"", ""hatlotthitran"", ""tthatlot""]"
1658,14,125,Huyện,4135.0,Xã,Tỉnh Sơn La,Huyện Mai Sơn,Xã Hát Lót,Sơn La,tinhsonla,...,hatlot,NaN,NaN,NaN,False,False,True,"[""tinhsonla"", ""sonla""]","[""huyenmaison"", ""maison""]","[""hatlotxa"", ""xahatlot"", ""xhatlot""]"
1820,15,139,Huyện,4585.0,Thị trấn,Tỉnh Yên Bái,Huyện Trạm Tấu,Thị trấn Trạm Tấu,Yên Bái,tinhyenbai,...,tramtau,NaN,NaN,NaN,False,False,True,"[""tinhyenbai"", ""yenbai""]","[""huyentramtau"", ""tramtau""]","[""thitrantramtau"", ""tramtauthitran"", ""tttramtau""]"
1825,15,139,Huyện,4600.0,Xã,Tỉnh Yên Bái,Huyện Trạm Tấu,Xã Trạm Tấu,Yên Bái,tinhyenbai,...,tramtau,NaN,NaN,NaN,False,False,True,"[""tinhyenbai"", ""yenbai""]","[""huyentramtau"", ""tramtau""]","[""xatramtau"", ""tramtauxa"", ""xtramtau""]"
2345,20,187,Huyện,6466.0,Thị trấn,Tỉnh Lạng Sơn,Huyện Chi Lăng,Thị trấn Chi Lăng,Lạng Sơn,tinhlangson,...,chilang,NaN,NaN,NaN,False,False,True,"[""tinhlangson"", ""langson""]","[""huyenchilang"", ""chilang""]","[""thitranchilang"", ""chilangthitran"", ""ttchilang""]"
2363,20,187,Huyện,6523.0,Xã,Tỉnh Lạng Sơn,Huyện Chi Lăng,Xã Chi Lăng,Lạng Sơn,tinhlangson,...,chilang,NaN,NaN,NaN,False,False,True,"[""tinhlangson"", ""langson""]","[""huyenchilang"", ""chilang""]","[""chilangxa"", ""xachilang"", ""xchilang""]"


In [27]:
# Create keywords
for col in unit_cols:
    level = level_map[col]
    df[f"{col}Keywords"] = df.apply(lambda row: create_keywords(row, level=level), axis=1)

In [28]:
# Province map
df_province = df[['provinceKey', 'provinceKeywords', 'province', 'provinceShort']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE = {}
for _, row in df_province.iterrows():
    DICT_PROVINCE[row['provinceKey']] = {
        'provinceKeywords': json.loads(row['provinceKeywords']),
        'province': row['province'],
        'provinceShort': row['provinceShort'],
    }


# District map
df_district = df[['provinceKey', 'provinceShortKey', 'districtKey', 'districtShortKey', 'districtKeywords', 'district', 'districtType', 'districtShort']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE_DISTRICT = {}
for _, province_row in df_province.iterrows():
    province_key = province_row['provinceKey']
    DICT_PROVINCE_DISTRICT[province_key] = {}

    df_district_filtered = df_district[df_district['provinceKey'] == province_key]

    for _, district_row in df_district_filtered.iterrows():
        DICT_PROVINCE_DISTRICT[province_key][district_row['districtKey']] = {
            'districtKeywords': json.loads(district_row['districtKeywords']) if pd.notnull(district_row['districtKeywords']) else [],
            'district': district_row['district'],
            'districtType': district_row['districtType'],
            'districtShort': district_row['districtShort'],
        }


# Unique district to province map
province_short_keys = df['provinceShortKey'].unique().tolist()
for index, row in df_district.iterrows():
    district_short_key = row['districtShortKey']
    left_district_short_keys = df_district.loc[df_district.index != index, 'districtShortKey'].tolist()
    if district_short_key not in province_short_keys and district_short_key not in left_district_short_keys:
        df_district.loc[index, 'districtUnique'] = True
df_district['districtUnique'].fillna(False, inplace=True)
df_district_unique = df_district[df_district['districtUnique']==True]

DICT_UNIQUE_DISTRICT_PROVINCE = {}
for _, row in df_district_unique.iterrows():
    DICT_UNIQUE_DISTRICT_PROVINCE[row['districtKey']] = {
        'districtKeywords': json.loads(row['districtKeywords']),
        'provinceKey': row['provinceKey']
    }

In [35]:
# Ward map
df_ward = df[['provinceKey', 'districtKey', 'wardKey', 'wardKeywords', 'ward', 'wardShort', 'wardType', 'wardKeyDuplicated']].drop_duplicates().reset_index(drop=True)

df_ward_no_accented = df_ward[df_ward['wardKeyDuplicated']==False]
df_ward_accented = df_ward[df_ward['wardKeyDuplicated']==True]

def build_province_district_ward_dict(df):
    DICT_PROVINCE_DISTRICT_WARD = {}

    for province_key, province_group in df.groupby('provinceKey'):
        DICT_PROVINCE_DISTRICT_WARD[province_key] = {}

        for district_key, district_group in province_group.groupby('districtKey'):
            DICT_PROVINCE_DISTRICT_WARD[province_key][district_key] = {}

            for _, row in district_group.iterrows():
                ward_key = row['wardKey']
                DICT_PROVINCE_DISTRICT_WARD[province_key][district_key][ward_key] = {
                    'wardKeywords': json.loads(row['wardKeywords']) if pd.notnull(row['wardKeywords']) else [],
                    'ward': row['ward'],
                    'wardShort': row['wardShort'],
                    'wardType': row['wardType'],
                }

    return DICT_PROVINCE_DISTRICT_WARD


DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED = build_province_district_ward_dict(df_ward_no_accented)
DICT_PROVINCE_DISTRICT_WARD_ACCENTED = build_province_district_ward_dict(df_ward_accented)

In [36]:
pickle_data = {
    'DICT_PROVINCE': DICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT': DICT_PROVINCE_DISTRICT,
    'DICT_UNIQUE_DISTRICT_PROVINCE': DICT_UNIQUE_DISTRICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED,
    'DICT_PROVINCE_DISTRICT_WARD_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_ACCENTED
}

In [38]:
DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED

{'thanhphocantho': {'huyencodo': {'xatrungan': {'wardKeywords': ['xatrungan',
     'trungan'],
    'ward': 'Xã Trung An',
    'wardShort': 'Trung An',
    'wardType': 'Xã'},
   'xatrungthanh': {'wardKeywords': ['xatrungthanh', 'trungthanh'],
    'ward': 'Xã Trung Thạnh',
    'wardShort': 'Trung Thạnh',
    'wardType': 'Xã'},
   'xathanhphu': {'wardKeywords': ['xathanhphu', 'thanhphu'],
    'ward': 'Xã Thạnh Phú',
    'wardShort': 'Thạnh Phú',
    'wardType': 'Xã'},
   'xatrunghung': {'wardKeywords': ['xatrunghung', 'trunghung'],
    'ward': 'Xã Trung Hưng',
    'wardShort': 'Trung Hưng',
    'wardType': 'Xã'},
   'thitrancodo': {'wardKeywords': ['thitrancodo', 'codo'],
    'ward': 'Thị trấn Cờ Đỏ',
    'wardShort': 'Cờ Đỏ',
    'wardType': 'Thị trấn'},
   'xathoihung': {'wardKeywords': ['xathoihung', 'thoihung'],
    'ward': 'Xã Thới Hưng',
    'wardShort': 'Thới Hưng',
    'wardType': 'Xã'},
   'xadonghiep': {'wardKeywords': ['xadonghiep', 'donghiep'],
    'ward': 'Xã Đông Hiệp',
    

In [37]:
DICT_PROVINCE_DISTRICT_WARD_ACCENTED

{'thanhphohue': {'thixahuongtra': {'phườnghươngvân': {'wardKeywords': ['phườnghươngvân',
     'hươngvân'],
    'ward': 'Phường Hương Vân',
    'wardShort': 'Hương Vân',
    'wardType': 'Phường'},
   'phườnghươngvăn': {'wardKeywords': ['phườnghươngvăn', 'hươngvăn'],
    'ward': 'Phường Hương Văn',
    'wardShort': 'Hương Văn',
    'wardType': 'Phường'}}},
 'tinhangiang': {'huyenphutan': {'xãphúthạnh': {'wardKeywords': ['xãphúthạnh',
     'phúthạnh'],
    'ward': 'Xã Phú Thạnh',
    'wardShort': 'Phú Thạnh',
    'wardType': 'Xã'},
   'xãphúthành': {'wardKeywords': ['xãphúthành', 'phúthành'],
    'ward': 'Xã Phú Thành',
    'wardShort': 'Phú Thành',
    'wardType': 'Xã'}}},
 'tinhbinhphuoc': {'huyenlocninh': {'xãlộcthạnh': {'wardKeywords': ['xãlộcthạnh',
     'lộcthạnh'],
    'ward': 'Xã Lộc Thạnh',
    'wardShort': 'Lộc Thạnh',
    'wardType': 'Xã'},
   'xãlộcthành': {'wardKeywords': ['xãlộcthành', 'lộcthành'],
    'ward': 'Xã Lộc Thành',
    'wardShort': 'Lộc Thành',
    'wardType': 'Xã

In [39]:
with open(BASE_DIR / 'vietnamadminunits/data/parser_63.json', 'w') as f:
    json.dump(pickle_data, f)